<a href="https://colab.research.google.com/github/SergeyB81/Portfolio-DS/blob/main/Car%20price/useful_stuff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! gdown  --id 1Ym8EaWePP17RvrweKhWzCJmITXLgqvCZ

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1Ym8EaWePP17RvrweKhWzCJmITXLgqvCZ
To: /content/churn-modeling.zip
100% 268k/268k [00:00<00:00, 45.1MB/s]


In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('/content/churn-modeling.zip')

In [4]:
df.head(5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


# **Обучающая, валидационная и тестовая выборка**

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
train, test = train_test_split(df,train_size=0.6,random_state=42,stratify=df['Exited'])

In [7]:
val, test = train_test_split(test,train_size=0.5,random_state=42,stratify=test['Exited'])

# **Обучение модели**

In [8]:
! pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 10.2 MB/s eta 0:00:00


In [9]:
from catboost import CatBoostClassifier

In [10]:
df.dtypes

,0
RowNumber,int64
CustomerId,int64
Surname,object
CreditScore,int64
Geography,object
Gender,object
Age,int64
Tenure,int64
Balance,float64
NumOfProducts,int64


In [11]:
df.select_dtypes(include='object')

,Surname,Geography,Gender
0,Hargrave,France,Female
1,Hill,Spain,Female
2,Onio,France,Female
3,Boni,France,Female
4,Mitchell,Spain,Female
...,...,...,...
9995,Obijiaku,France,Male
9996,Johnstone,France,Male
9997,Liu,France,Female
9998,Sabbatini,Germany,Male


In [12]:
df.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [13]:
X = ['CustomerId', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary']

cat_features = ['Geography','Gender']

y = ['Exited']

In [14]:
from catboost import Pool

train_data = Pool(data=train[X],
                  label=train[y],
                  cat_features=cat_features
                  )

valid_data = Pool(data=val[X],
                  label=val[y],
                  cat_features=cat_features
                  )

test_data = Pool(data=test[X],
                  label=test[y],
                  cat_features=cat_features
                  )



In [15]:
params = {'verbose':100,
          'eval_metric':'AUC',
          'loss_function':'Logloss',
          'random_seed':42,
          'learning_rate':0.01}

In [16]:
model = CatBoostClassifier(**params)

In [17]:
model.fit(train_data,eval_set=valid_data)

0:	test: 0.8338545	best: 0.8338545 (0)	total: 71.2ms	remaining: 1m 11s
100:	test: 0.8725090	best: 0.8725090 (100)	total: 1.9s	remaining: 16.9s
200:	test: 0.8784656	best: 0.8784656 (200)	total: 3.85s	remaining: 15.3s
300:	test: 0.8812676	best: 0.8812676 (300)	total: 6.03s	remaining: 14s
400:	test: 0.8823868	best: 0.8824376 (390)	total: 8.58s	remaining: 12.8s
500:	test: 0.8834491	best: 0.8834522 (498)	total: 11s	remaining: 11s
600:	test: 0.8834737	best: 0.8836554 (564)	total: 13.8s	remaining: 9.17s
700:	test: 0.8837847	best: 0.8838694 (685)	total: 15.6s	remaining: 6.65s
800:	test: 0.8834383	best: 0.8838694 (685)	total: 16.4s	remaining: 4.06s
900:	test: 0.8829318	best: 0.8838694 (685)	total: 17.1s	remaining: 1.88s
999:	test: 0.8822744	best: 0.8838694 (685)	total: 17.9s	remaining: 0us

bestTest = 0.8838694083
bestIteration = 685

Shrink model to first 686 iterations.


In [18]:
n_iters = model.best_iteration_ + 1

In [19]:
n_iters

686

In [20]:
params = {'iterations':n_iters,
          'verbose':100,
          'eval_metric':'AUC',
          'loss_function':'Logloss',
          'random_seed':42,
          'learning_rate':0.01}

In [21]:
model = CatBoostClassifier(**params)

In [22]:
train_full = pd.concat([train,val])

In [23]:
train_full_data = Pool(data=train_full[X],
                  label=train_full[y],
                  cat_features=cat_features
                  )

In [24]:
model.fit(train_full_data)

0:	total: 9.55ms	remaining: 6.54s
100:	total: 947ms	remaining: 5.49s
200:	total: 1.84s	remaining: 4.44s
300:	total: 2.74s	remaining: 3.51s
400:	total: 3.65s	remaining: 2.6s
500:	total: 4.54s	remaining: 1.67s
600:	total: 5.44s	remaining: 769ms
685:	total: 6.21s	remaining: 0us


In [25]:
test['y_score_no_cross_val'] = model.predict_proba(test_data)[:,1]

In [26]:
from sklearn.metrics import roc_auc_score

In [27]:
roc_auc_score(test['Exited'],test['y_score_no_cross_val'])

np.float64(0.8735839074822127)

# **Кросс валидация**

In [28]:
from catboost import cv

In [29]:
params = {'verbose':100,
          'eval_metric':'AUC',
          'loss_function':'Logloss',
          'random_seed':42,
          'learning_rate':0.01}

In [32]:
cv_data = cv(
    params = params,
    pool = train_full_data,
    fold_count=5,
    shuffle=True,
    partition_random_seed = 0,
    stratified=False,
    verbose=False
)

Training on fold [0/5]

bestTest = 0.866360207
bestIteration = 881

Training on fold [1/5]

bestTest = 0.8716621864
bestIteration = 998

Training on fold [2/5]

bestTest = 0.8695412245
bestIteration = 540

Training on fold [3/5]

bestTest = 0.8795945701
bestIteration = 942

Training on fold [4/5]

bestTest = 0.8586794872
bestIteration = 750



In [34]:
cv_data

,iterations,test-AUC-mean,test-AUC-std,test-Logloss-mean,test-Logloss-std,train-Logloss-mean,train-Logloss-std
0,0,0.818479,0.009480,0.685020,0.000209,0.684939,0.000262
1,1,0.823848,0.010334,0.677216,0.000374,0.677055,0.000503
2,2,0.831819,0.009265,0.668989,0.000472,0.668799,0.000751
3,3,0.834559,0.008771,0.662008,0.001326,0.661753,0.000689
4,4,0.834483,0.008218,0.654599,0.001429,0.654270,0.000898
...,...,...,...,...,...,...,...
995,995,0.868703,0.007809,0.329986,0.016873,0.284097,0.003644
996,996,0.868715,0.007781,0.329972,0.016875,0.284057,0.003631
997,997,0.868718,0.007804,0.329972,0.016876,0.284003,0.003636
998,998,0.868725,0.007777,0.329969,0.016855,0.283943,0.003613


In [43]:
cv_data[cv_data['test-AUC-mean'] == cv_data['test-AUC-mean'].max()]

,iterations,test-AUC-mean,test-AUC-std,test-Logloss-mean,test-Logloss-std,train-Logloss-mean,train-Logloss-std
741,741,0.868822,0.007457,0.330818,0.017005,0.297428,0.004326


In [56]:
n_iters = cv_data[cv_data['test-AUC-mean'] == cv_data['test-AUC-mean'].max()]['iterations'].values[0]

In [57]:
n_iters

np.int64(741)

In [58]:

params = {'iterations':n_iters,
          'verbose':100,
          'eval_metric':'AUC',
          'loss_function': 'Logloss',
          'random_seed':42,
          'learning_rate':0.01}

In [59]:
model = CatBoostClassifier(**params)

In [60]:
model.fit(train_full_data)

0:	total: 21ms	remaining: 15.6s
100:	total: 1.52s	remaining: 9.64s
200:	total: 2.4s	remaining: 6.45s
300:	total: 3.3s	remaining: 4.82s
400:	total: 4.21s	remaining: 3.56s
500:	total: 5.09s	remaining: 2.44s
600:	total: 5.99s	remaining: 1.4s
700:	total: 7.22s	remaining: 412ms
740:	total: 8.06s	remaining: 0us


In [66]:
test['y_score_cross_val'] = model.predict_proba(test_data)[:,1]

In [67]:
roc_auc_score(test['Exited'],test['y_score_cross_val'])

np.float64(0.8737196364315009)

In [68]:
len(train_full)

8000

# **Подбор гиперпараметров**

In [69]:
model.get_all_params()

{'nan_mode': 'Min',
 'eval_metric': 'AUC',
 'combinations_ctr': ['Borders:CtrBorderCount=15:CtrBorderType=Uniform:TargetBorderCount=1:TargetBorderType=MinEntropy:Prior=0/1:Prior=0.5/1:Prior=1/1',
  'Counter:CtrBorderCount=15:CtrBorderType=Uniform:Prior=0/1'],
 'iterations': 741,
 'sampling_frequency': 'PerTree',
 'fold_permutation_block': 0,
 'leaf_estimation_method': 'Newton',
 'random_score_type': 'NormalWithModelSizeDecrease',
 'counter_calc_method': 'SkipTest',
 'grow_policy': 'SymmetricTree',
 'penalties_coefficient': 1,
 'boosting_type': 'Plain',
 'model_shrink_mode': 'Constant',
 'feature_border_type': 'GreedyLogSum',
 'ctr_leaf_count_limit': 18446744073709551615,
 'bayesian_matrix_reg': 0.10000000149011612,
 'one_hot_max_size': 2,
 'eval_fraction': 0,
 'force_unit_auto_pair_weights': False,
 'l2_leaf_reg': 3,
 'random_strength': 1,
 'rsm': 1,
 'boost_from_average': False,
 'max_ctr_complexity': 4,
 'model_size_reg': 0.5,
 'simple_ctr': ['Borders:CtrBorderCount=15:CtrBorderType=

In [70]:

params = {'verbose':100,
          'eval_metric':'AUC',
          'loss_function': 'Logloss',
          'random_seed':42}

In [71]:
model = CatBoostClassifier(**params)

In [72]:
grid = {'learning_rate':[0.01, 0.1],
        'depth':[5,6]}

In [73]:
result = model.grid_search(grid,train_full_data,verbose=False)

0:	test: 0.7811641	best: 0.7811641 (0)	total: 21ms	remaining: 21s
100:	test: 0.8435462	best: 0.8435769 (99)	total: 1.44s	remaining: 12.8s
200:	test: 0.8508923	best: 0.8508923 (200)	total: 2.41s	remaining: 9.6s
300:	test: 0.8547897	best: 0.8547897 (300)	total: 3.62s	remaining: 8.41s
400:	test: 0.8569385	best: 0.8569385 (400)	total: 4.83s	remaining: 7.21s
500:	test: 0.8576410	best: 0.8576410 (500)	total: 5.7s	remaining: 5.67s
600:	test: 0.8585538	best: 0.8585538 (600)	total: 6.48s	remaining: 4.3s
700:	test: 0.8590923	best: 0.8591487 (694)	total: 7.25s	remaining: 3.09s
800:	test: 0.8593590	best: 0.8595564 (749)	total: 8.01s	remaining: 1.99s
900:	test: 0.8593282	best: 0.8595564 (749)	total: 8.8s	remaining: 967ms
999:	test: 0.8593897	best: 0.8595974 (986)	total: 9.58s	remaining: 0us

bestTest = 0.8595974359
bestIteration = 986

Metric AUC is not calculated on train by default. To calculate this metric on train, add hints=skip_train~false to metric parameters.
0:	test: 0.7811641	best: 0.7811

In [76]:
result['params']

{'depth': 5, 'learning_rate': 0.1}

In [77]:
pd.DataFrame(result['cv_results'])['test-AUC-mean'].max()


0.8689660773549619